In [1]:
import pandas as pd
from tqdm import tqdm
import time
import re
import requests
import difflib
import operator
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz 

In [2]:
import diff_match_patch as dmp_module
dmp_module.Match_Distance = 50
dmp = dmp_module.diff_match_patch()

ModuleNotFoundError: No module named 'diff_match_patch'

In [3]:
def calc_ratio_t(a, b):
    diff = dmp.diff_main(a, b)
    dleven = dmp.diff_levenshtein(diff)
    return 1 - (dleven / max(len(aw), len(bw)))

In [4]:
def calc_match(text, pattern):
    loc = dmp.match_main(text, pattern, 0)
    print(loc)
    return loc >= 0

In [5]:
calc_ratio_t("BUTYROSPERMUM PRKII BUTTER asd", "BUTYROSPERMUM PARKII BUTTER")

NameError: name 'dmp' is not defined

In [6]:
calc_match("asd BUTYROSPERMUM PARKII BUTTER", "BUTYROSPERMUM PRKII (Shea) BUTTER")

NameError: name 'dmp' is not defined

In [7]:
from fuzzywuzzy import fuzz 
fuzz.ratio("BUTYROSPERMUM PRKII (Shea) BUTTER", "BUTYROSPERMUM PARKII BUTTER") / 100

0.87

In [8]:


class FuzzyDict(dict):
    "Provides a dictionary that performs fuzzy lookup"
    def __init__(self, items = None, cutoff = .6):
        """Construct a new FuzzyDict instance

        items is an dictionary to copy items from (optional)
        cutoff is the match ratio below which mathes should not be considered
        cutoff needs to be a float between 0 and 1 (where zero is no match
        and 1 is a perfect match)"""
        super(FuzzyDict, self).__init__()

        if items:
            self.update(items)
        self.cutoff =  cutoff

        # short wrapper around some super (dict) methods
        self._dict_contains = lambda key: \
            super(FuzzyDict,self).__contains__(key)

        self._dict_getitem = lambda key: \
            super(FuzzyDict,self).__getitem__(key)

    def _search(self, lookfor, stop_on_first = False):
        """Returns the value whose key best matches lookfor

        if stop_on_first is True then the method returns as soon
        as it finds the first item
        """

        # if the item is in the dictionary then just return it
        if self._dict_contains(lookfor):
            return True, lookfor, self._dict_getitem(lookfor), 1

        # set up the fuzzy matching tool
        #ratio_calc = difflib.SequenceMatcher()
        #ratio_calc.set_seq1(lookfor)

        # test each key in the dictionary
        best_ratio = 0
        best_match = None
        best_key = None
        for key in self:

            # if the current key is not a string
            # then we just skip it
            if not isinstance(key, str):
                continue

            # we get an error here if the item to look for is not a
            # string - if it cannot be fuzzy matched and we are here
            # this it is defintely not in the dictionary
            try:
            # calculate the match value
                ratio = ratio_calc.ratio()
            except TypeError:
                break

            # if this is the best ratio so far - save it and the value
            if ratio > best_ratio:
                best_ratio = ratio
                best_key = key
                best_match = self._dict_getitem(key)

            if stop_on_first and ratio >= self.cutoff:
                break

        return (
            best_ratio >= self.cutoff,
            best_key,
            best_match,
            best_ratio)


    def __contains__(self, item):
        "Overides Dictionary __contains__ to use fuzzy matching"
        if self._search(item, True)[0]:
            return True
        else:
            return False

    def __getitem__(self, lookfor):
        "Overides Dictionary __getitem__ to use fuzzy matching"
        matched, key, item, ratio = self._search(lookfor)

        if not matched:
            raise KeyError(
                "'%s'. closest match: '%s' with ratio %.3f"%
                    (str(lookfor), str(key), ratio))

        return item

In [9]:
def fuzzy_match_ingredients(ing_list, fuzdict):
    match_dict = {}
    for ing in tqdm(ing_list):
        if ing in match_dict.keys():
            continue
        upper_ing = ing.upper()
        if fuzdict.__contains__(upper_ing):
            match_dict[ing] = fuzdict[upper_ing]
    
    return match_dict

In [10]:
def cosing_lookup(ing_list, match_dict, desc, restrictions, functions):
    res = []
    for item in ing_list:
        key = match_dict.get(item, "")
        if key == "":
            continue
        description = desc.get(key, "unknown")
        restrictn = restrictions.get(key, "unknown")
        functn = functions.get(key, "unknown")
        res.extend([[key, description, restrictn, functn]])
    
    return res

In [11]:
def create_cosing_dict(ingredient_df, col_name):
    col_dict = {name.strip(): row[col_name] for (idx, row) in ingredient_df.iterrows() for name in row['INCI name'].split('/')} # .split('/')

    return col_dict

In [15]:
dfcosing = pd.read_csv('../ingredient_cosing_37309.csv')
# dfcosing.drop(columns=['Unnamed: 0'], inplace=True)
dfcosing

,cosing_ref_no,ingredient_name,description,restriction,functions
0,94753.0,DISODIUM TETRAMETHYLHEXADECENYLCYSTEINE FORMYL...,Disodium Tetramethylhexadecenylcysteine Formyl...,NaN,SKIN PROTECTING
1,96229.0,ASTROCARYUM VULGARE SEED BUTTER,Astrocaryum Vulgare Seed Butter is the fat obt...,NaN,"SKIN CONDITIONING, SKIN CONDITIONING - EMOLLIENT"
2,89177.0,BARLEY SH-POLYPEPTIDE-17,Barley sh-Polypeptide-17 is a single chain rec...,NaN,"HAIR CONDITIONING, SKIN CONDITIONING"
3,98580.0,DAUCUS CAROTA SATIVA LEAF EXTRACT,Daucus Carota Sativa (Carrot) Leaf Extract is ...,NaN,SKIN CONDITIONING - MISCELLANEOUS
4,89078.0,GOSSYPIUM HIRSUTUM SEED EXTRACT,Gossypium Hirsutum Seed Extract is the extrac...,NaN,SKIN CONDITIONING
...,...,...,...,...,...
37304,90014.0,PHELLODENDRON AMURENSE BARK,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"
37305,90014.0,LONICERA JAPONICA FLOWER,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"
37306,90014.0,CHAENOMELES SINENSIS FRUIT,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"
37307,90014.0,CAMELLIA SINENSIS LEAF,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"


In [213]:
# dfi3 = pd.read_csv('Database/ingredient_inci_1570.csv')
ingnames_dict = {name.strip(): name.strip() for name in dfcosing['INCI name']}
# print(ingnames_dict)
fd = FuzzyDict(ingnames_dict, cutoff = .5)
print(len(fd))

29908


In [212]:
len(ingnames_dict)

29908

In [206]:
# SOME TESTING SHIT
fuzzy_match_ingredients(["BUTYROSPERMUM (Shea) BUTTER"],fd)

100%|██████████| 1/1 [00:00<00:00,  3.10it/s]


{'BUTYROSPERMUM (Shea) BUTTER': 'BUTYROSPERMUM PARKII BUTTER'}

In [11]:
# ingnames_dict = {name.strip(): name.strip() for name in df_ingredient['INCI name']}
# fd = FuzzyDict(ingnames_dict, cutoff = .55)

# # Compare product ingredient list and database
# # match_dict = find_matching_ingredient(ing_list, rating, 0.55)
# match_dict = fuzzy_match_ingredients(ing_list, fd)
# print(match_dict)

## **Load EWG ingredient list**

In [195]:
def clean_string_name(string):
    text = string.replace(' Size: 4 oz. * USDA Certified Organic Ingredient ** None remains after saponifying oils into soap and glycerin', '')
    pattern = "[\|\*\_\'\{}&]".format('"')
    regex1 = re.compile('\[.*]')
    regex2 = re.compile('\[.*}')
    text = re.sub(pattern, "", text)
    text = re.sub(regex1, "", text)
    text = re.sub(regex2, "", text)
    text = text.strip('\x0c')
    text = text.strip(' ')
    text = text.upper()
    text = text.replace('INACTIVE INGREDIENTS:', '')
    text = text.replace('ACTIVE INGREDIENTS:', '')

    return text

In [109]:
def clean_string(string):
    pattern = "[\|\*\_\'\{}&]".format('"')
    regex = re.compile('\\\S*')
    text = re.sub(pattern, "", string)
    text = re.sub(",, ", ", ", text)
    text = re.sub(regex, " ", text)
    text = text.split(':')[1]
#     text = text.split('.')[0]
    text = text.strip()
#     split = [x.strip(' ').lower() for x in text.split(',')]

    return text

def string_to_list(text):
    pattern = "[\|\*\_\'\{}]".format('"')
    text = re.sub(pattern, "", text)
    text = text.replace('Aqua/Water/Eau', 'Aqua')
#     split = [x.strip(' ').lower() for x in text.split([\\.,]\\s)split('.')]
    split = [x.strip(' ').lower().replace('aqua/water/eau', 'aqua') for x in re.split("[,.]", text)]
    return split

In [120]:
df_sun = pd.read_csv('../Database/ewg_sun_full_1565.csv')

In [121]:
df_sun

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list
0,Sun,After Sun Product,https://www.ewg.org/skindeep/products/707220-A...,ATTITUDE,"little ones Aftersun melt-in Gel, Calendula, T...",verified,https://static.ewg.org/skindeep_images/7072/70...,"Aqua, butyrospermum parkii (shea) butter, dica..."
1,Sun,After Sun Product,https://www.ewg.org/skindeep/products/671517-A...,AROMATICA,95% Organic Aloe Vera Gel,verified,https://static.ewg.org/skindeep_images/6715/67...,"ALOE BARBADENSIS LEAF EXTRACT, PROPANEDIOL, RO..."
2,Sun,After Sun Product,https://www.ewg.org/skindeep/products/823174-O...,Olita,"After Sun Body Serum, Fragrance Free",verified,https://static.ewg.org/skindeep_images/8231/82...,"Cocos Nucifera (Coconut) Oil, Carthamus Tincto..."
3,Sun,After Sun Product,https://www.ewg.org/skindeep/products/710965-V...,Vika's Essentials,'Prodigal Sun' Oil Blend,verified,https://static.ewg.org/skindeep_images/7109/71...,"Sesame Oil (Sesamum indicum)*, Coconut Oil (Co..."
4,Sun,After Sun Product,https://www.ewg.org/skindeep/products/899338-O...,Olita,"After Sun Body Serum, Soothing Citrus",verified,https://static.ewg.org/skindeep_images/8993/89...,"Cocos Nucifera (Coconut) Oil, Carthamus Tincto..."
...,...,...,...,...,...,...,...,...
1560,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/929454-C...,COOLA,"Classic Body Sunscreen Spray, Peach Blossom, S...",10,https://static.ewg.org/skindeep_images/9294/92...,"Active Ingredients: Avobenzone 2.9%, Octisalat..."
1561,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/929145-C...,CVS,"Health Ultra Protection Sun Lotion, SPF 100",10,https://static.ewg.org/skindeep_images/9291/92...,"Active Ingredients: Avobenzone 3%, Homosalate ..."
1562,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/929320-A...,Australian Gold,"Botanical Natural Sunscreen Spray, SPF 70",10,https://static.ewg.org/skindeep_images/9293/92...,"Active Ingredients: Avobenzone 3%, Homosalate ..."
1563,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/928941-B...,Banana Boat,"Ultra Sport Sunscreen Lotion, SPF 100",10,https://static.ewg.org/skindeep_images/9289/92...,"Active Ingredients: Avobenzone 2.5%, Octocryle..."


In [138]:
df_sun['product_score'].value_counts()

03          428
04          243
02          193
07          173
05          166
01          165
06          135
verified     23
08           20
10           18
09            1
Name: product_score, dtype: int64

In [130]:
example = df_sun[df_sun['ingredient_list'] != 'No ingredient list']['ingredient_list'].to_list()

In [131]:
len(example)

1565

In [152]:
samplea = [name.upper().strip() for name in clean_string_name(example[447]).split(',')]
samplea

['ZINC OXIDE 14%',
 'WATER',
 'CAPRYLIC/CAPRIC TRIGLYCERIDE',
 'COCO-CAPRYLATE',
 'TRIMETHYLSILOXYSILICATE',
 'ISOPROPYL PALMITATE',
 'DIMETHICONE/VINYL DIMETHICONE CROSSPOLYMER',
 'POLYGLYCERYL-4 DIISOSTEARATE/POLYHYDROXYSTEARATE/SEBACATE',
 'OCTYLDODECYL OLEATE',
 'ETHYLHEXYL PALMITATE',
 'ISOPROPYL MYRISTATE',
 'DIMETHICONE',
 'MAGNESIUM SULFATE',
 'BEESWAX',
 'GLUCONOLACTONE',
 'SODIUM HYALURONATE',
 'LECITHIN',
 'RIBES NIGRUM (BLACK CURRANT) SEED OIL',
 'ROSMARINUS OFFICINALIS (ROSEMARY) LEAF EXTRACT',
 'SOLIDAGO VIRGAUREA (GOLDENROD) EXTRACT',
 'POLYHYDROXYSTEARIC ACID',
 'POLYGLYCERYL-3 POLYRICINOLEATE',
 'ISOSTEARIC ACID',
 'GLYCERIN',
 'PROPANEDIOL',
 'SODIUM BENZOATE',
 'ISOEICOSANE',
 'HYDROGENATED CASTOR OIL',
 'TETRAHEXYLDECYL ASCORBATE',
 'OCTYLDODECANOL',
 'OCTYLDODECYL STEAROYL STEARATE',
 'ALCOHOL',
 'PAEONIA SUFFRUTICOSA ROOT EXTRACT',
 'PRUNUS AMYGDALUS DULCIS (SWEET ALMOND) OIL',
 'CITRUS AURANTIUM BERGAMIA (BERGAMOT) FRUIT OIL',
 'CITRUS AURANTIUM DULCIS (ORANGE) P

In [144]:
# match_dict = find_matching_ingredient(ing_list, rating, 0.55)
match_dict = fuzzy_match_ingredients(sample, fd)
print(match_dict)

100%|██████████| 47/47 [00:49<00:00,  1.06s/it]

{'ZINC OXIDE 14%': 'ZINC OXIDE', 'WATER': 'WATER', 'CAPRYLIC/CAPRIC TRIGLYCERIDE': 'CAPRIC TRIGLYCERIDE', 'COCO-CAPRYLATE': 'COCO-CAPRYLATE', 'TRIMETHYLSILOXYSILICATE': 'TRIMETHYLSILOXYSILICATE', 'ISOPROPYL PALMITATE': 'ISOPROPYL PALMITATE', 'DIMETHICONE/VINYL DIMETHICONE CROSSPOLYMER': 'DIVINYLDIMETHICONE CROSSPOLYMER', 'POLYGLYCERYL-4 DIISOSTEARATE/POLYHYDROXYSTEARATE/SEBACATE': 'POLYGLYCERYL-2 DIPOLYHYDROXYSTEARATE', 'OCTYLDODECYL OLEATE': 'OCTYLDODECYL OLEATE', 'ETHYLHEXYL PALMITATE': 'ETHYLHEXYL PALMITATE', 'ISOPROPYL MYRISTATE': 'ISOPROPYL MYRISTATE', 'DIMETHICONE': 'DIMETHICONE', 'MAGNESIUM SULFATE': 'MAGNESIUM SULFATE', 'BEESWAX': 'BEESWAX', 'GLUCONOLACTONE': 'GLUCONOLACTONE', 'SODIUM HYALURONATE': 'SODIUM HYALURONATE', 'LECITHIN': 'LECITHIN', 'RIBES NIGRUM (BLACK CURRANT) SEED OIL': 'BLACK CURRANT SEED OIL', 'ROSMARINUS OFFICINALIS (ROSEMARY) LEAF EXTRACT': 'ROSMARINUS OFFICINALIS LEAF EXTRACT', 'SOLIDAGO VIRGAUREA (GOLDENROD) EXTRACT': 'SOLIDAGO VIRGAUREA EXTRACT', 'POLYHYDRO

In [44]:
for o in match_dict.values():
    print(o)

SEA WATER
STEARYL ACRYLATE COPOLYMER
PROPYLENE GLYCOL
LAURETH-21
PENTYLENE GLYCOL
PEG-40 HYDROGENATED CASTOR OIL
PHENOXYETHANOL
AMMONIUM ACRYLATES COPOLYMER
CAPRYLYL GLYCOL
PPG-2-DECETH-30
SODIUM DEHYDROACETATE
SODIUM LAURYL SULFATE
DISODIUM LAURETH SULFOSUCCINATE
C16-24 ALKYL C16-24 ACIDATE


In [145]:
df_sun

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list
0,Sun,After Sun Product,https://www.ewg.org/skindeep/products/707220-A...,ATTITUDE,"little ones Aftersun melt-in Gel, Calendula, T...",verified,https://static.ewg.org/skindeep_images/7072/70...,"Aqua, butyrospermum parkii (shea) butter, dica..."
1,Sun,After Sun Product,https://www.ewg.org/skindeep/products/671517-A...,AROMATICA,95% Organic Aloe Vera Gel,verified,https://static.ewg.org/skindeep_images/6715/67...,"ALOE BARBADENSIS LEAF EXTRACT, PROPANEDIOL, RO..."
2,Sun,After Sun Product,https://www.ewg.org/skindeep/products/823174-O...,Olita,"After Sun Body Serum, Fragrance Free",verified,https://static.ewg.org/skindeep_images/8231/82...,"Cocos Nucifera (Coconut) Oil, Carthamus Tincto..."
3,Sun,After Sun Product,https://www.ewg.org/skindeep/products/710965-V...,Vika's Essentials,'Prodigal Sun' Oil Blend,verified,https://static.ewg.org/skindeep_images/7109/71...,"Sesame Oil (Sesamum indicum)*, Coconut Oil (Co..."
4,Sun,After Sun Product,https://www.ewg.org/skindeep/products/899338-O...,Olita,"After Sun Body Serum, Soothing Citrus",verified,https://static.ewg.org/skindeep_images/8993/89...,"Cocos Nucifera (Coconut) Oil, Carthamus Tincto..."
...,...,...,...,...,...,...,...,...
1560,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/929454-C...,COOLA,"Classic Body Sunscreen Spray, Peach Blossom, S...",10,https://static.ewg.org/skindeep_images/9294/92...,"Active Ingredients: Avobenzone 2.9%, Octisalat..."
1561,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/929145-C...,CVS,"Health Ultra Protection Sun Lotion, SPF 100",10,https://static.ewg.org/skindeep_images/9291/92...,"Active Ingredients: Avobenzone 3%, Homosalate ..."
1562,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/929320-A...,Australian Gold,"Botanical Natural Sunscreen Spray, SPF 70",10,https://static.ewg.org/skindeep_images/9293/92...,"Active Ingredients: Avobenzone 3%, Homosalate ..."
1563,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/928941-B...,Banana Boat,"Ultra Sport Sunscreen Lotion, SPF 100",10,https://static.ewg.org/skindeep_images/9289/92...,"Active Ingredients: Avobenzone 2.5%, Octocryle..."


In [46]:
df_skin[df_skin['ingredient_list'] == 'No ingredient list']

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list
7173,Makeup,Lip Gloss,https://www.ewg.org/skindeep/products/920738-S...,Saint,"Amplified Lip Lacquer, Barolo",01,https://static.ewg.org/skindeep_images/9207/92...,No ingredient list
7432,Makeup,Lip Gloss,https://www.ewg.org/skindeep/products/938211-U...,Uoma,"Beauty Boss Gloss Liquid Marble, Ambition",04,https://static.ewg.org/skindeep_images/9382/93...,No ingredient list


In [197]:
def compare_ingredient(x):
    x = clean_string_name(x)
    ing_list = [name.strip() for name in x.split(',')]
    match_dict = fuzzy_match_ingredients(ing_list, fd)
    res = [i for i in match_dict.values()]
#     print(sample)
#     print(res)
    
    return res

In [55]:
df_skin.to_csv('ewg_makeup_full_after_processing_10527.csv', index=False)

In [150]:
df_testing = df_sun.iloc[:5, :]
df_testing

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list
0,Sun,After Sun Product,https://www.ewg.org/skindeep/products/707220-A...,ATTITUDE,"little ones Aftersun melt-in Gel, Calendula, T...",verified,https://static.ewg.org/skindeep_images/7072/70...,"Aqua, butyrospermum parkii (shea) butter, dica..."
1,Sun,After Sun Product,https://www.ewg.org/skindeep/products/671517-A...,AROMATICA,95% Organic Aloe Vera Gel,verified,https://static.ewg.org/skindeep_images/6715/67...,"ALOE BARBADENSIS LEAF EXTRACT, PROPANEDIOL, RO..."
2,Sun,After Sun Product,https://www.ewg.org/skindeep/products/823174-O...,Olita,"After Sun Body Serum, Fragrance Free",verified,https://static.ewg.org/skindeep_images/8231/82...,"Cocos Nucifera (Coconut) Oil, Carthamus Tincto..."
3,Sun,After Sun Product,https://www.ewg.org/skindeep/products/710965-V...,Vika's Essentials,'Prodigal Sun' Oil Blend,verified,https://static.ewg.org/skindeep_images/7109/71...,"Sesame Oil (Sesamum indicum)*, Coconut Oil (Co..."
4,Sun,After Sun Product,https://www.ewg.org/skindeep/products/899338-O...,Olita,"After Sun Body Serum, Soothing Citrus",verified,https://static.ewg.org/skindeep_images/8993/89...,"Cocos Nucifera (Coconut) Oil, Carthamus Tincto..."


In [154]:
df_testing['new_ing_list'] = df_testing['ingredient_list'].apply(compare_ingredient)

  0%|          | 0/47 [00:00<?, ?it/s]


TypeError: super(type, obj): obj must be an instance or subtype of type

In [100]:
df_skin['new_ing_list'] = df_skin['ingredient_list'].apply(compare_ingredient)

100%|██████████| 16/16 [00:00<00:00, 23391.03it/s]


100%|██████████| 41/41 [00:00<00:00, 89147.99it/s]


100%|██████████| 23/23 [00:00<00:00, 55569.70it/s]


100%|██████████| 22/22 [00:00<00:00, 49318.38it/s]


100%|██████████| 7/7 [00:00<00:00, 21604.21it/s]


...

100%|██████████| 47/47 [00:02<00:00, 23.33it/s]


100%|██████████| 29/29 [00:00<00:00, 69032.25it/s]


100%|██████████| 24/24 [00:00<00:00, 74126.14it/s]


100%|██████████| 32/32 [00:00<00:00, 58203.70it/s]


100%|██████████| 6/6 [00:00<00:00, 18.45it/s]


In [203]:
df_skin.to_csv('ewg_makeup_done_processing_ced.csv', index=False)

In [77]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [201]:
df_testing[['ingredient_list', 'new_ing_list']]

,ingredient_list,new_ing_list
0,"Aqua/Water/Eau, Ethyl Macadamiate, Butyrospermum Parkii (Shea) Butter*, Rhus Verniciflua Peel Wax, Mica, Titanium Dioxide (CI 77891), Glycerin*, Potassium Cetyl Phosphate, Cetearyl Olivate, Sorbitan Olivate, Xylitylglucoside, Hydroxyacetophenone, Prunus Armeniaca (Apricot) Kernel Oil*, Anhydroxylitol, Carbomer, 1,2-Hexanediol, Caprylyl Glycol, Xylitol, Sodium Phytate, Tocopherol*, Aloe Barbadensis Leaf Juice*, Helianthus Annuus (Sunflower) Seed Oil*, Arginine, Citric Acid, Iron Oxides (CI 77491, CI 77499).","[CAVIAR WATER, ETHYL MACADAMIATE, BUTYROSPERMUM PARKII BUTTER, RHUS VERNICIFLUA PEEL WAX, MICA, TITANIUM DIOXIDE, GLYCERIN, POTASSIUM CETYL PHOSPHATE, CETEARYL OLIVATE, SORBITAN OLIVATE, XYLITYLGLUCOSIDE, HYDROXYACETOPHENONE, PRUNUS ARMENIACA KERNEL OIL, ANHYDROXYLITOL, CARBOMER, 12, 1,2-HEXANEDIOL, CAPRYLYL GLYCOL, XYLITOL, SODIUM PHYTATE, TOCOPHEROL, ALOE BARBADENSIS LEAF JUICE, HELIANTHUS ANNUUS SEED OIL, ARGININE, CITRIC ACID, IRON OXIDES, CI 77499]"
1,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Stearate, Lauroyl Lysine, Caprylic/Capric Triglyceride, Tocopheryl Acetate (Vitamin E), Ascorbyl Palmitate (Vitamin C), Camellia Sinensis (White Tea) Leaf Extract, Aspalathus Linearis (Rooibos Red Tea) Leaf Extract, Punica Granatum (Pomegranate) Extract, Laminaria Ochroleuca (Sea Kelp) Extract, Aloe Barbadensis (Aloe Vera) Leaf Extract, Sambucus Nigra (Elderberry) Fruit Extract, Phenoxyethanol, May contain: Iron Oxides, Titanium Dioxide","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC STEARATE, LAUROYL LYSINE, CAPRIC TRIGLYCERIDE, TOCOPHERYL ACETATE, ASCORBYL PALMITATE, CAMELLIA SINENSIS LEAF EXTRACT, ASPALATHUS LINEARIS LEAF EXTRACT, PUNICA GRANATUM PEEL EXTRACT, LAMINARIA OCHROLEUCA EXTRACT, ALOE BARBADENSIS LEAF EXTRACT, SAMBUCUS NIGRA FRUIT EXTRACT, PHENOXYETHANOL, IRON OXIDES, TITANIUM DIOXIDE]"
2,"Aqua/Water/Eau, Ethyl Macadamiate, Mica, Butyrospermum Parkii (Shea) Butter*, Titanium Dioxide (CI 77891), Rhus Verniciflua Peel Wax, Glycerin*, Potassium Cetyl Phosphate, Synthetic Fluorphlogopite, Cetearyl Olivate, Sorbitan Olivate, Xylitylglucoside, Hydroxyacetophenone, Prunus Armeniaca (Apricot) Kernel Oil*, Anhydroxylitol, Carbomer, 1,2-Hexanediol, Caprylyl Glycol, Xylitol, Sodium Phytate, Tocopherol*, Tin Oxide, Aloe Barbadensis Leaf Juice*, Helianthus Annuus (Sunflower) Seed Oil*, Arginine, Citric Acid.","[CAVIAR WATER, ETHYL MACADAMIATE, MICA, BUTYROSPERMUM PARKII BUTTER, TITANIUM DIOXIDE, RHUS VERNICIFLUA PEEL WAX, GLYCERIN, POTASSIUM CETYL PHOSPHATE, SYNTHETIC FLUORPHLOGOPITE, CETEARYL OLIVATE, SORBITAN OLIVATE, XYLITYLGLUCOSIDE, HYDROXYACETOPHENONE, PRUNUS ARMENIACA KERNEL OIL, ANHYDROXYLITOL, CARBOMER, 12, 1,2-HEXANEDIOL, CAPRYLYL GLYCOL, XYLITOL, SODIUM PHYTATE, TOCOPHEROL, TIN OXIDE, ALOE BARBADENSIS LEAF JUICE, HELIANTHUS ANNUUS SEED OIL, ARGININE, CITRIC ACID]"
3,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Stearate, Lauroyl Lysine, Caprylic/Capric Triglyceride, Tocopheryl Acetate (Vitamin E), Ascorbyl Palmitate (Vitamin C), Camellia Sinensis (White Tea) Leaf Extract, Aspalathus Linearis (Rooibos Red Tea) Leaf Extract, Punica Granatum (Pomegranate) Extract, Laminaria Ochroleuca (Sea Kelp) Extract, Aloe Barbadensis (Aloe Vera) Leaf Extract, Sambucus Nigra (Elderberry) Fruit Extract, Phenoxyethanol, May contain: Calcium Sodium Borosilicate, Carmine, Iron Oxides, Manganese Violet, Tin Oxide, Titanium Dioxide","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC STEARATE, LAUROYL LYSINE, CAPRIC TRIGLYCERIDE, TOCOPHERYL ACETATE, ASCORBYL PALMITATE, CAMELLIA SINENSIS LEAF EXTRACT, ASPALATHUS LINEARIS LEAF EXTRACT, PUNICA GRANATUM PEEL EXTRACT, LAMINARIA OCHROLEUCA EXTRACT, ALOE BARBADENSIS LEAF EXTRACT, SAMBUCUS NIGRA FRUIT EXTRACT, PHENOXYETHANOL, CALCIUM SODIUM BOROSILICATE, CALAMINE, IRON OXIDES, MANGANESE DIOXIDE, TIN OXIDE, TITANIUM DIOXIDE]"
4,"Caprylic/Capric Triglyceride, Jojoba Esters, Silica, Butyrospermum Parkii (Shea) Butter, Euphorbia Cerifera (Candelilla) Wax/Cire de candelilla, Mica,

In [75]:
df_testing

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list,new_ing_list
0,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/929942-C...,C'est Moi,"Dream Beams Cream Highlighter, Aura",verified,https://static.ewg.org/skindeep_images/9299/92...,"Aqua/Water/Eau, Ethyl Macadamiate, Butyrosperm...","[TEA-LAURATE, ETHYL MACADAMIATE, BUTYROSPERMUM..."
1,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704637-M...,Mineral Fusion,"Bronzer, Luster Bronzer duo",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
2,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/929943-C...,C'est Moi,"Dream Beams Cream Highlighter, Moonstruck",verified,https://static.ewg.org/skindeep_images/9299/92...,"Aqua/Water/Eau, Ethyl Macadamiate, Mica, Butyr...","[TEA-LAURATE, ETHYL MACADAMIATE, MICA, BUTYROS..."
3,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704641-M...,Mineral Fusion,"Blush/Bronzer Duo, Rio Blonzer",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
4,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820241-B...,Beautycounter,"Illuminating Cream Highlighter, Bronze Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, B..."
5,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704639-M...,Mineral Fusion,"Bronzer, Sparkle Bronzer",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
6,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/646346-R...,Rejuva Minerals,"Bronzer, Caribbean",verified,https://static.ewg.org/skindeep_images/6463/64...,"Boron Nitride,Â Kaolin, Â *CoperniciaÂ Cerifer...","[BORON NITRIDE, KAOLIN, COPERNICIA CERIFERA WA..."
7,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820244-B...,Beautycounter,"Illuminating Cream Highlighter, Rose Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, M..."
8,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820242-B...,Beautycounter,"Illuminating Cream Highlighter, Pearl Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, C..."
9,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/782243-W...,W3LL PEOPLE,Bio Bronzer Baked Powder,verified,https://static.ewg.org/skindeep_images/7822/78...,"""MICA, ZEA MAYS STARCH / ZEA MAYS (CORN) STARC...","[MICA, ARACHIS HYPOGAEA (PEANUT) SPROUT EXTRA..."


In [67]:
df_testing[['ingredient_list', 'new_ing_list']]

,ingredient_list,new_ing_list
0,"Aqua/Water/Eau, Ethyl Macadamiate, Butyrosperm...","[ETHYL MACADAMIATE, BUTYROSPERMUM PARKII BUTTE..."
1,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
2,"Aqua/Water/Eau, Ethyl Macadamiate, Mica, Butyr...","[ETHYL MACADAMIATE, MICA, BUTYROSPERMUM PARKII..."
3,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
4,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, B..."
5,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
6,"Boron Nitride,Â Kaolin, Â *CoperniciaÂ Cerifer...","[BORON NITRIDE, KAOLIN, COPERNICIA CERIFERA WA..."
7,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, M..."
8,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, C..."
9,"""MICA, ZEA MAYS STARCH / ZEA MAYS (CORN) STARC...","[MICA, OCTYLDODECANOL, PENTYLENE GLYCOL, HECTO..."


In [68]:
df_testing

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list,new_ing_list
0,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/929942-C...,C'est Moi,"Dream Beams Cream Highlighter, Aura",verified,https://static.ewg.org/skindeep_images/9299/92...,"Aqua/Water/Eau, Ethyl Macadamiate, Butyrosperm...","[ETHYL MACADAMIATE, BUTYROSPERMUM PARKII BUTTE..."
1,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704637-M...,Mineral Fusion,"Bronzer, Luster Bronzer duo",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
2,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/929943-C...,C'est Moi,"Dream Beams Cream Highlighter, Moonstruck",verified,https://static.ewg.org/skindeep_images/9299/92...,"Aqua/Water/Eau, Ethyl Macadamiate, Mica, Butyr...","[ETHYL MACADAMIATE, MICA, BUTYROSPERMUM PARKII..."
3,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704641-M...,Mineral Fusion,"Blush/Bronzer Duo, Rio Blonzer",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
4,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820241-B...,Beautycounter,"Illuminating Cream Highlighter, Bronze Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, B..."
5,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/704639-M...,Mineral Fusion,"Bronzer, Sparkle Bronzer",verified,https://static.ewg.org/skindeep_images/7046/70...,"Mica, Octyldodecyl Stearoyl Stearate, Zinc Ste...","[MICA, OCTYLDODECYL STEAROYL STEARATE, ZINC ST..."
6,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/646346-R...,Rejuva Minerals,"Bronzer, Caribbean",verified,https://static.ewg.org/skindeep_images/6463/64...,"Boron Nitride,Â Kaolin, Â *CoperniciaÂ Cerifer...","[BORON NITRIDE, KAOLIN, COPERNICIA CERIFERA WA..."
7,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820244-B...,Beautycounter,"Illuminating Cream Highlighter, Rose Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, M..."
8,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/820242-B...,Beautycounter,"Illuminating Cream Highlighter, Pearl Glow",verified,https://static.ewg.org/skindeep_images/8202/82...,"Caprylic/Capric Triglyceride, Jojoba Esters, S...","[CAPRIC TRIGLYCERIDE, JOJOBA ESTERS, SILICA, C..."
9,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/782243-W...,W3LL PEOPLE,Bio Bronzer Baked Powder,verified,https://static.ewg.org/skindeep_images/7822/78...,"""MICA, ZEA MAYS STARCH / ZEA MAYS (CORN) STARC...","[MICA, OCTYLDODECANOL, PENTYLENE GLYCOL, HECTO..."


## **SequenceMatcher**

In [83]:
def find_matching_ingredient(my_ingredients, rating_dict, thresh=0.5):
    ''' my_ingredients: list of product's ingredients
        Loop thru each ingredient in the ingredient list of the products
        then check if that ingredient appears in our ingredient list
        Calculate match_metric using SequenceMatcher and return the highest score and the best match
        Compare the match_metric with thresh > append to match_dict
    '''
    match_dict = {}
    for ingredient in tqdm(my_ingredients):
        if ingredient in match_dict.keys():
            continue
        match_metric = {key: SequenceMatcher(None, key, ingredient).ratio() for key in rating_dict.keys()}
        best_match, best_metric = max(match_metric.items(), key=operator.itemgetter(1)) # What is operator.itemgetter(1)
        if best_metric > thresh:
            match_dict[ingredient] = best_match
        else:
            match_dict[ingredient] = 'unknown'
    return match_dict

In [89]:
def compare_ingredient(x):
    x = clean_string_name(x)
    sample = [name.strip() for name in x.split(',')]
    match_dict = find_matching_ingredient(sample, fd)
    res = [i for i in match_dict.values()]
#     print(sample)
#     print(res)
    
    return res

In [90]:
df_testing['new_ing_list'] = df_testing['ingredient_list'].apply(compare_ingredient)

100%|██████████| 9/9 [00:21<00:00,  2.38s/it]
2020-10-17 20:28:07,599 [16154] WARNING  py.warnings:110: [JupyterRequire] /Users/macbookpro/miniconda3/envs/cs_ftmle/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



In [91]:
df_skin['product_score'].value_counts()

04          1561
01          1347
03          1225
05          1222
02          1203
verified    1132
06           369
07           102
08            36
09            10
10             1
Name: product_score, dtype: int64

## **FuzzyPuzzy**

In [29]:


class FuzzyDict(dict):
    "Provides a dictionary that performs fuzzy lookup"
    def __init__(self, items = None, cutoff = .6):
        """Construct a new FuzzyDict instance

        items is an dictionary to copy items from (optional)
        cutoff is the match ratio below which mathes should not be considered
        cutoff needs to be a float between 0 and 1 (where zero is no match
        and 1 is a perfect match)"""
        super(FuzzyDict, self).__init__()

        if items:
            self.update(items)
        self.cutoff =  cutoff

        # short wrapper around some super (dict) methods
        self._dict_contains = lambda key: \
            super(FuzzyDict,self).__contains__(key)

        self._dict_getitem = lambda key: \
            super(FuzzyDict,self).__getitem__(key)

    def _search(self, lookfor, stop_on_first = False):
        """Returns the value whose key best matches lookfor

        if stop_on_first is True then the method returns as soon
        as it finds the first item
        """

        # if the item is in the dictionary then just return it
        if self._dict_contains(lookfor):
            return True, lookfor, self._dict_getitem(lookfor), 1

        # set up the fuzzy matching tool
        #ratio_calc = difflib.SequenceMatcher()
        #ratio_calc.set_seq1(lookfor)

        # test each key in the dictionary
        best_ratio = 0
        best_match = None
        best_key = None
        for key in self:

            # if the current key is not a string
            # then we just skip it
            if not isinstance(key, str):
                continue

            # we get an error here if the item to look for is not a
            # string - if it cannot be fuzzy matched and we are here
            # this it is defintely not in the dictionary
            try:
            # calculate the match value
                ratio = fuzz.ratio(lookfor, key)/100
            except TypeError:
                break

            # if this is the best ratio so far - save it and the value
            if ratio > best_ratio:
                best_ratio = ratio
                best_key = key
                best_match = self._dict_getitem(key)

            if stop_on_first and ratio >= self.cutoff:
                break

        return (
            best_ratio >= self.cutoff,
            best_key,
            best_match,
            best_ratio)


    def __contains__(self, item):
        "Overides Dictionary __contains__ to use fuzzy matching"
        if self._search(item, True)[0]:
            return True
        else:
            return False

    def __getitem__(self, lookfor):
        "Overides Dictionary __getitem__ to use fuzzy matching"
        matched, key, item, ratio = self._search(lookfor)

        if not matched:
            raise KeyError(
                "'%s'. closest match: '%s' with ratio %.3f"%
                    (str(lookfor), str(key), ratio))

        return item

In [30]:
def fuzzy_match_ingredients(ing_list, fuzdict):
    match_dict = {}
    for ing in tqdm(ing_list):
        if ing in match_dict.keys():
            continue
        upper_ing = ing.upper()
        if fuzdict.__contains__(upper_ing):
            match_dict[ing] = fuzdict[upper_ing]
    
    return match_dict

In [31]:
def cosing_lookup(ing_list, match_dict, desc, restrictions, functions):
    res = []
    for item in ing_list:
        key = match_dict.get(item, "")
        if key == "":
            continue
        description = desc.get(key, "unknown")
        restrictn = restrictions.get(key, "unknown")
        functn = functions.get(key, "unknown")
        res.extend([[key, description, restrictn, functn]])
    
    return res

In [32]:
def create_cosing_dict(ingredient_df, col_name):
    col_dict = {name.strip(): row[col_name] for (idx, row) in ingredient_df.iterrows() for name in row['INCI name'].split('/')} # .split('/')

    return col_dict

In [58]:
def clean_string_name(string):
    water = ['WATER(AQUA)', 'AQUA', 'EAU', 'AQUA/WATER/EAU', 'AQUA / WATER / EAU', 'PURIFIED WATER', 'DISTILLED WATER']
    text = string.replace(' Size: 4 oz. * USDA Certified Organic Ingredient ** None remains after saponifying oils into soap and glycerin', '')
    pattern = "[\|\*\_\'\{}&]".format('"')
    regex1 = re.compile('\[.*]')
    regex2 = re.compile('\[.*}')
    text = re.sub(pattern, "", text)
    text = re.sub(regex1, "", text)
    text = re.sub(regex2, "", text)
    text = text.strip('\x0c')
    text = text.strip(' ')
    text = text.upper()
    text = text.replace('INACTIVE INGREDIENTS:', '')
    text = text.replace('ACTIVE INGREDIENTS:', '')
    text = text.replace('INGREDIENTS:', '')
    
    return text

In [64]:
def remove_water(string):
    water = ['WATER (AQUA)', 'AQUA', 'EAU', 'AQUA/WATER/EAU', 'AQUA / WATER / EAU', 
             'PURIFIED WATER', 'DISTILLED WATER', 'D.I. WATER', 'AQUA (WATER)']
    if string in water:
        string = 'WATER'
    text = string.strip()
    
    return text

In [82]:
def compare_ingredient(x):
    x = clean_string_name(x)
    ing_list = [remove_water(name) for name in x.split(',')]
    match_dict = fuzzy_match_ingredients(ing_list, fd)
    res = [i for i in match_dict.values()]
#     print(sample)
#     print(res)
    
    return res

In [36]:
# dfi3 = pd.read_csv('Database/ingredient_inci_1570.csv')
ingnames_dict = {name.strip(): name.strip() for name in dfcosing['ingredient_name']}
# print(ingnames_dict)
fd = FuzzyDict(ingnames_dict, cutoff = .6)
print(len(fd))

29908


In [24]:
fuzzy_match_ingredients(["BUTYROSPERMUM (Shea) BUTTER"],fd)

100%|██████████| 1/1 [00:00<00:00,  4.81it/s]


{'BUTYROSPERMUM (Shea) BUTTER': 'BUTYROSPERMUM PARKII BUTTER'}

In [68]:
df_hair = pd.read_csv('../EWG/ewg_hair_full_4908.csv')
df_skin = pd.read_csv('../EWG/ewg_skin_full_8208.csv')
df_makeup = pd.read_csv('../EWG/ewg_makeup_full_10527.csv')
df_sun = pd.read_csv('../EWG/ewg_sun_full_1565.csv')

In [83]:
testing_sun = df_hair.loc[50:70, :]

In [84]:
testing_sun['new_ing_list'] = testing_sun['ingredient_list'].apply(compare_ingredient)

100%|██████████| 45/45 [00:05<00:00,  8.34it/s]
2020-10-20 01:51:17,866 [33512] WARNING  py.warnings:110: [JupyterRequire] /Users/macbookpro/miniconda3/envs/cs_ftmle/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



In [86]:
pd.set_option('display.max_colwidth', -1)
testing_sun[['ingredient_list', 'new_ing_list']]

2020-10-20 01:52:10,250 [33512] WARNING  py.warnings:110: [JupyterRequire] /Users/macbookpro/miniconda3/envs/cs_ftmle/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.



,ingredient_list,new_ing_list
50,"Water (Aqua), Sodium Lauroyl Methyl Isethionate, Cocamidopropyl Hydroxysultaine, Lactobacillus Ferment, Aloe Barbadensis Leaf Juice*, Sodium Cocoyl Apple Amino Acids, Glycerin (Vegetable), Olea Europaea (Olive) Fruit Oil*, Cucumis Sativus (Cucumber) Oil, Brassica Oleracea Italica (Broccoli) Seed Oil, Sodium Chloride, Caprylic/Capric Triglyceride, Citrus Aurantium Dulcis (Orange) Peel Oil*, Vanilla Planifolia Fruit Extract","[WATER, SODIUM LAUROYL METHYL ISETHIONATE, COCAMIDOPROPYL HYDROXYSULTAINE, LACTOBACILLUS FERMENT, ALOE BARBADENSIS LEAF JUICE, SODIUM COCOYL APPLE AMINO ACIDS, FERMENTED VEGETABLE, OLEA EUROPAEA FRUIT OIL, CUCUMIS SATIVUS OIL, BRASSICA OLERACEA ITALICA SEED OIL, SODIUM CHLORIDE, CAPRIC TRIGLYCERIDE, CITRUS AURANTIUM DULCIS PEEL OIL, VANILLA PLANIFOLIA FRUIT EXTRACT]"
51,"Water, Decyl Polyglucose, Lauryl Polyglucose, Cocos Nucifera (Coconut) Oil, Chondrus Crispus (Carrageenan) Extract, Pelargonium Graveolens (Rose Geranium) Infusion, Pelargonium Graveolens (Rose Geranium) Infusion, Lavandula Officinalis (Lavender) Infusion, Symphytum Officinale (Comfrey) Root Infusion, Fragaria (Strawberry) Leaf Infusion, Calendula Officinalis (Pot Marigold) Infusion, Avena Sativa (Oat) Straw, Chamomilla Recutita (Matricaria) Flower Extract, Hippophae Rhamnoides (Sea Buckthorn), Calendula Officinalis (Pot Marigold), Citrus Paradisi (Grapefruit) Peel Extract","[WATER, DECYL GLUCOSIDE, LAURYL GLUCOSIDE, COCOS NUCIFERA OIL, CHONDRUS CRISPUS EXTRACT, PELARGONIUM GRAVEOLENS STEM LEAF OIL, LAVANDULA OFFICINALIS FLOWER OIL, SYMPHYTUM OFFICINALE ROOT EXTRACT, CALENDULA OFFICINALIS FLOWER OIL, AVENA SATIVA STARCH, CHAMOMILLA RECUTITA (MATRICARIA) FLOWER, HIPPOPHAE RHAMNOIDES EXTRACT, CALENDULA OFFICINALIS FLOWER OIL, CITRUS PARADISI FRUIT EXTRACT]"
52,"WATER, SODIUM LAURYLGLUCOSIDES HYDROXYPROPYLSULFONATE, SODIUM METHYL COCOYL TAURATE, COCAMIDOPROPYL HYDROXYSULTAINE, SODIUM BIS-HYDROXYETHYLGLYCINATE COCO-GLUCOSIDES CROSSPOLYMER, COCAMIDOPROPYL PG-DIMONIUM CHLORIDE PHOSPHATE, GLUCONOLACTONE, SODIUM STEAROYL LACTYLATE, LACTOBACILLUS FERMENT, AVENA SATIVA (OAT) KERNEL EXTRACT*, ALTHAEA OFFICINALIS ROOT EXTRACT (MARSHMALLOW)*, CHENOPODIUM QUINOA SEED EXTRACT (QUINOA)*, SODIUM BENZOATE","[WATER, SODIUM LAURYLGLUCOSIDES HYDROXYPROPYLSULFONATE, SODIUM METHYL COCOYL TAURATE, COCAMIDOPROPYL HYDROXYSULTAINE, SODIUM BIS-HYDROXYETHYLGLYCINATE COCO-GLUCOSIDES CROSSPOLYMER, COCAMIDOPROPYL PG-DIMONIUM CHLORIDE PHOSPHATE, GLUCONOLACTONE, SODIUM STEAROYL LACTYLATE, LACTOBACILLUS FERMENT, AVENA SATIVA KERNEL EXTRACT, ALTHAEA OFFICINALIS ROOT EXTRACT, CHENOPODIUM QUINOA SEED EXTRACT, SODIUM BENZOATE]"
53,"20215028V01 - INGREDIENTS: WATER, DECYL GLUCOSIDE, POLYGLYCERYL-10 CAPRYLATE/CAPRATE, NIACINAMIDE, COCOGLYCERIDES, CETYL HYDROXYETHYLCELLULOSE, CERAMIDE NP, CERAMIDE AP, CERAMIDE EOP, CARBOMER, GLYCERYL OLEATE, SODIUM BENZOATE, SODIUM LAUROYL LACTYLATE, CHOLESTEROL, PHENOXYETHANOL, COCO-GLUCOSIDE, TOCOPHERYL ACETATE, CITRIC ACID, HYDROLYZED HYALURONIC ACID, XANTHAN GUM, PHYTOSPHINGOSINE, ETHYLHEXYLGLYCERIN","[DECYL GLUCOSIDE, POLYGLYCERYL-10 CAPRYLATE, NIACINAMIDE, COCOGLYCERIDES, CETYL HYDROXYETHYLCELLULOSE, CERAMIDE NP, CERAMIDE AP, CERAMIDE EOP, CARBOMER, GLYCERYL OLEATE, SODIUM BENZOATE, SODIUM LAUROYL LACTYLATE, CHOLESTEROL, PHENOXYETHANOL, COCO-GLUCOSIDE, TOCOPHERYL ACETATE, CITRIC ACID, HYDROLYZED HYALURONIC ACID, XANTHAN GUM, PHYTOSPHINGOSINE, ETHYLHEXYLGLYCERIN]"
54,"Water,\nSodium lauryl glucosides hydroxypropylsulfonate,\nGlycerin,\nDecyl glucoside,\nGlyceryl caprylate,\nXanthan gum,\nSodium chloride,\nSodium levulinate,\nGlyceryl undecylenate,\nLavandula angustifolia (Lavender) oil,\nCalendula officinalis (Marigold ) flower extract,\nCaprylic/capric triglyceride,\nCoriandrum sativum (Coriander) seed oil,\nEuphrasia officinalis (Eyebright) extract,\nCitrus aurantium bergamia (Bergamot) peel oil (Furocoumarin-free),\nCitric acid,\nCananga odorata (Ylang ylang) flower oil,\nPelargonium graveolens (Geranium) f

In [87]:
df_hair['new_ing_list'] = df_hair['ingredient_list'].apply(compare_ingredient)
df_hair.to_csv('ewg_hair_products_transformed_4908.csv', index=False)

100%|██████████| 26/26 [00:05<00:00,  5.20it/s]


100%|██████████| 25/25 [00:05<00:00,  4.42it/s]


100%|██████████| 28/28 [00:03<00:00,  9.25it/s]


100%|██████████| 26/26 [00:00<00:00, 26.76it/s]


100%|██████████| 57/57 [00:07<00:00,  7.85it/s]


...

100%|██████████| 16/16 [00:00<00:00, 90.55it/s]


100%|██████████| 27/27 [00:02<00:00, 12.18it/s]


100%|██████████| 18/18 [00:00<00:00, 53.13it/s]


100%|██████████| 11/11 [00:00<00:00, 13.01it/s]


100%|██████████| 29/29 [00:00<00:00, 44.93it/s]


In [88]:
df_skin['new_ing_list'] = df_skin['ingredient_list'].apply(compare_ingredient)
df_skin.to_csv('ewg_skin_products_transformed_8208.csv', index=False)

100%|██████████| 8/8 [00:01<00:00,  6.45it/s]


100%|██████████| 11/11 [00:00<00:00, 12.89it/s]


100%|██████████| 11/11 [00:01<00:00,  6.02it/s]


100%|██████████| 23/23 [00:02<00:00,  9.42it/s]


100%|██████████| 5/5 [00:00<00:00, 41363.94it/s]


...

100%|██████████| 13/13 [00:01<00:00,  7.67it/s]


100%|██████████| 14/14 [00:00<00:00, 14.75it/s]


100%|██████████| 31/31 [00:00<00:00, 43.05it/s]


100%|██████████| 5/5 [00:01<00:00,  4.32it/s]


100%|██████████| 9/9 [00:01<00:00,  5.98it/s]


In [89]:
df_sun['new_ing_list'] = df_sun['ingredient_list'].apply(compare_ingredient)
df_sun.to_csv('ewg_sun_products_transformed_1565.csv', index=False)

100%|██████████| 13/13 [00:00<00:00, 18.22it/s]


100%|██████████| 25/25 [00:01<00:00, 17.70it/s]


100%|██████████| 51/51 [00:01<00:00, 45.80it/s]


100%|██████████| 47/47 [00:01<00:00, 34.66it/s]


100%|██████████| 9/9 [00:00<00:00,  9.31it/s]


100%|██████████| 25/25 [00:01<00:00, 20.27it/s]


100%|██████████| 18/18 [00:01<00:00, 16.35it/s]


100%|██████████| 34/34 [00:01<00:00, 19.22it/s]


100%|██████████| 17/17 [00:01<00:00,  9.22it/s]


100%|██████████| 20/20 [00:02<00:00,  9.65it/s]


In [90]:
df_makeup['new_ing_list'] = df_makeup['ingredient_list'].apply(compare_ingredient)
df_makeup.to_csv('ewg_makeup_products_transformed_10527.csv', index=False)

100%|██████████| 10/10 [00:00<00:00, 39.86it/s]


100%|██████████| 15/15 [00:01<00:00, 13.01it/s]


100%|██████████| 14/14 [00:01<00:00,  9.13it/s]


100%|██████████| 11/11 [00:00<00:00, 12.17it/s]


100%|██████████| 22/22 [00:00<00:00, 25.61it/s]


...

100%|██████████| 19/19 [00:03<00:00,  6.21it/s]


100%|██████████| 51/51 [00:04<00:00, 10.93it/s]


100%|██████████| 55/55 [00:05<00:00, 10.28it/s]


100%|██████████| 16/16 [00:01<00:00,  9.55it/s]


100%|██████████| 6/6 [00:00<00:00, 11.20it/s]


In [92]:
df_oc = pd.read_csv('../EWG/ewg_oralcare_full_937.csv')
df_oc['new_ing_list'] = df_oc['ingredient_list'].apply(compare_ingredient)
df_oc.to_csv('ewg_oralcare_products_transformed_937.csv', index=False)

100%|██████████| 6/6 [00:00<00:00, 10.43it/s]


100%|██████████| 23/23 [00:02<00:00, 10.27it/s]


100%|██████████| 70/70 [00:04<00:00, 14.63it/s]


100%|██████████| 6/6 [00:00<00:00, 29.44it/s]


100%|██████████| 22/22 [00:04<00:00,  5.21it/s]


100%|██████████| 17/17 [00:01<00:00, 14.23it/s]


In [93]:
df_frag = pd.read_csv('../EWG/ewg_frarance_full_484.csv')
df_frag['new_ing_list'] = df_frag['ingredient_list'].apply(compare_ingredient)
df_frag.to_csv('ewg_fragrance_products_transformed_484.csv', index=False)

100%|██████████| 8/8 [00:00<00:00, 44.98it/s]


100%|██████████| 12/12 [00:01<00:00,  6.35it/s]


100%|██████████| 25/25 [00:01<00:00, 23.63it/s]


In [230]:
# df_sun['new_ing_list'] = df_sun['ingredient_list'].apply(compare_ingredient)

100%|██████████| 25/25 [00:02<00:00, 11.85it/s]


100%|██████████| 47/47 [00:02<00:00, 20.65it/s]


100%|██████████| 25/25 [00:01<00:00, 14.24it/s]


100%|██████████| 34/34 [00:02<00:00, 12.85it/s]


100%|██████████| 20/20 [00:02<00:00,  6.91it/s]


In [231]:
# df_sun.to_csv('ewg_sun_products_transformed_1565.csv', index=False)

In [235]:
# pd.set_option('display.max_colwidth', -1)
# df_skin[['ingredient_list', 'new_ing_list']]


2020-10-18 10:30:38,499 [16154] WARNING  py.warnings:110: [JupyterRequire] /Users/macbookpro/miniconda3/envs/cs_ftmle/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.



,ingredient_list,new_ing_list
0,"Saponified Organic Extra Virgin Olive Oil (Sodium Olivate), Distilled Water, Cinnamomum Zeylanicum (Cinnamon) Leaf Oil,","[SODIUM PEG-7 OLIVE OIL CARBOXYLATE, STEM WATER, CINNAMOMUM ZEYLANICUM LEAF OIL]"
1,"Safflower* and/or Sunflower* Oils, Coconut* Oil, Palm Fruit* Oil, Shea Butter*, Water, Sodium Hydroxide**, Rosemary Extract. Size: 4 oz. * USDA Certified Organic Ingredient ** None remains after saponifying oils into soap and glycerin","[SAFFLOWER FLOWER, COCONUT ACID, PALM) FRUIT, SHEA BUTTERATE, WATER, SODIUM HYDROXIDE, ROSA EXTRACT]"
2,"Saponified Organic Sunflower Oil (Sodium Sunflower Seedate), Saponified Organic Palm Kernel Oil (Sodium Palm Kernelate), Saponified Organic Coconut Oil (Sodium Cocoate), Distilled Water, Butyrospermum Parkii (Shea Butter), Avena Sativa (Oat) Kernel Meal,","[POTASSIUM SUNFLOWERSEEDATE, SODIUM PALM KERNELOYL ISETHIONATE, PROPYLENE GLYCOL DICOCOATE, STEM WATER, BUTYROSPERMUM PARKII BUTTER, AVENA SATIVA KERNEL MEAL]"
3,"Saponified Organic Sunflower Oil (Sodium Sunflower Seedate), Saponified Organic Palm Kernel Oil (Sodium Palm Kernelate), Saponified Organic Coconut Oil (Sodium Cocoate), Distilled Water, Organic Simmondsia Chinensis (Jojoba), Organic Butyrospermum Parkii (Shea Butter), Avena Sativa (Oat) Kernel Meal, Natural Sea Salt (Sodium Chloride),","[POTASSIUM SUNFLOWERSEEDATE, SODIUM PALM KERNELOYL ISETHIONATE, PROPYLENE GLYCOL DICOCOATE, STEM WATER, SIMMONDSIA CHINENSIS BUTTER, BUTYROSPERMUM PARKII BUTTER, AVENA SATIVA KERNEL MEAL, LAURALKONIUM CHLORIDE]"
4,"Saponified Expeller Pressed Babassu Oil (Orbignya Oleifera Oil), Saponified Organic Sunflower Oil (Sodium Sunflower Seedate), Distilled Water, Organic Simmondsia Chinensis (Jojoba), Lavandula Angustifolia (Lavender) Flower Essential Oil, Organic Avena Sativa (Oat) Kernel Meal, Cinnamomum Zeylanicum (Cinnamon) Leaf Oil,","[POTASSIUM SUNFLOWERSEEDATE, STEM WATER, SIMMONDSIA CHINENSIS BUTTER, LAVANDULA ANGUSTIFOLIA FLOWER OIL, AVENA SATIVA KERNEL MEAL, CINNAMOMUM ZEYLANICUM LEAF OIL]"
...,...,...
8203,"Maranta Arundinacea (Arrowroot) Root Powder, Cocos Nucifera (Coconut) Oil*, Sodium Bicarbonate**, Euphorbia Antisyphillitica (Candelilla Wax), Pimenta Racemosa (Bay) Oil, Cedrus Atlantica (Cedarwood) Oil , Abies Sibirica (Fir Needle) Oil","[MARANTA ARUNDINACEA ROOT POWDER, COCOS NUCIFERA OIL, SODIUM BICARBONATE, SYNTHETIC CANDELILLA WAX, PIMENTA RACEMOSA LEAF, CEDRUS ATLANTICA WOOD OIL, ABIES SIBIRICA NEEDLE OIL]"
8204,"MAGNESIUM HYDROXIDE, COCOS NUCIFERA (COCONUT) OIL, MARANTA ARUNDINACEA (ARROWROOT) POWDER, BUTYROSPERMUM PARKII (SHEA BUTTER), CAPRYLIC/CAPRIC TRIGLYCERIDE (FRACTIONATED COCONUT OIL), SIMMONDSIA CHINENSIS (JOJOBA) SEED OIL, EUPHORBIA CERIFERA (CANDELILLA) WAX, TOCOPHEROL (VITAMIN E, SUNFLOWER DERIVED), PELARGONIUM GRAVEOLENS (GERANIUM) ESSENTIAL OIL.","[MAGNESIUM HYDROXIDE, COCOS NUCIFERA OIL, MARANTA ARUNDINACEA ROOT POWDER, BUTYROSPERMUM PARKII BUTTER, CAPRIC TRIGLYCERIDE ESTERS, SIMMONDSIA CHINENSIS SEED OIL, EUPHORBIA CERIFERA CERA, TOCOPHERYL NICOTINATE, SUNFLOWER SEED, PELARGONIUM GRAVEOLENS STEM LEAF OIL]"
8205,"COCOS NUCIFERA (COCONUT OIL), EUPHORBIA CERIFERA (CANDELILLA) WAX, CERA ALBA (BEESWAX), ZEA MAYS (CORN STARCH), MARANTA ARUNDINACEA (ARROWROOT) POWDER), SODIUM BICARBONATE, BUTYROSPERMUM PARKII (SHEA) BUTTER, CITRUS AURANTUM DULCIS FRUIT OIL, CEDARWOOD OIL","[COCOS NUCIFERA OIL, EUPHORBIA CERIFERA CERA, STEARYL BEESWAX, ZEA MAYS STARCH, MARANTA ARUNDINACEA ROOT POWDER, SODIUM BICARBONATE, BUTYROSPERMUM PARKII BUTTER, CITRUS AURANTIUM DULCIS FLOWER OIL, WOOD OIL]"
8206,"CAPRYLIC/CAPRIC TRIGLYCERIDE, TAPIOCA STARCH,\nOZOKERITE, SODIUM BICARBONATE, MAGNESIUM\nHYDROXIDE, COCOS NUCIFERA (COCONUT) OIL,\nCYCLODEXTRIN, BUTYROSPERMUM PARKII\n(SHEA) BUTTER, DEXTROSE, LACTOBACILLUS ACIDOPHILUS,\nCUCUMIS SATIVUS (CUCUMBER) FRUIT EXTRACT, MENTHA\nSPICATA FLOWER/LEAF/STEM (SPEARMINT) EXTRACT,\nGERANIUM MACULATUM EXTRACT, CITRUS AURANTIUM\nAMARA (BITTER ORANGE) FLOWER OIL","[CAPRIC TRIGLYC